In [2]:
%run ../functions_scripts/database_functions.ipynb
%run ../functions_scripts/dataframe_cleaning_functions.ipynb

In [3]:

from pathlib import Path
from typing import List, Optional
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import text
from psycopg2.extras import execute_values
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

RAW_DIR = Path(os.getenv("RAW_DIR"))
CLEAN_DIR = Path(os.getenv("CLEAN_DIR"))
ADMIN_DB_URL = os.getenv("ADMIN_DB_URL")
DB_URL = os.getenv("DB_URL")



# 🧹 Nettoyage de la table Lieux 

In [4]:
# Etablir connection avec la base de données
engine = get_engine(DB_URL) 

In [5]:
#Afficher le nombre de lignes dans la table bronze.lieux_raw

query = "SELECT COUNT(*) FROM bronze.lieux_raw"
count= get_single_value(engine, query)
print(f"Nombre de lignes dans bronze.lieux_raw : {count}")
print('\n')
#Récupérer les données dans Dataframe df_lieux
df_lieux = get_df_from_table(engine, "bronze", "lieux_raw")

Nombre de lignes dans bronze.lieux_raw : 1427733




In [6]:
# Supprimer les doublons
print(f"Nombre de lignes avant suppression des doublons : {df_lieux.shape[0]}")
df_lieux = df_lieux.drop_duplicates()
print(f"Nombre de lignes après suppression des doublons : {df_lieux.shape[0]}")
#remplacer les valeurs nulles
remplacer_valeurs_nulles(df_lieux)
#Nettoyage sémantique 
#Traiter valeurs spéciales (-1, 00, NULL chaîne).
df_lieux =  traiter_valeurs_speciales(df_lieux)
df_lieux.head()

Nombre de lignes avant suppression des doublons : 1427733
Nombre de lignes après suppression des doublons : 475911


,identifiant_de_l_accident,categorie_route,voie,v1,v2,circulation,nombre_de_voies,voie_reservee,profil,pr,pr1,plan,largeur_terre_plein_central,largeur_de_la_chaussee,surface,infrastructure,situation,env1
0,201700033619,Voie Communale,0,None,None,A sens unique,2,None,Plat,None,None,En courbe à gauche,0,80,normale,Pont - autopont,None,NaN
1,201700048262,Autoroute,1,None,None,A sens unique,2,None,Plat,4,500,Partie rectiligne,0,70,normale,None,Sur chaussée,99
2,201700048288,Autoroute,86,None,None,A sens unique,2,None,Plat,8,0,Partie rectiligne,0,0,mouillée,None,Sur chaussée,99
3,201700048486,Autoroute,12,None,None,A sens unique,2,None,Plat,5,600,Partie rectiligne,0,0,normale,Souterrain - tunnel,Sur chaussée,NaN
4,201700048639,Route Nationale,118,None,None,A sens unique,2,None,Plat,5,350,Partie rectiligne,15,70,normale,None,Sur chaussée,99


In [7]:
#traiter valeur nulle de df_lieux
df_lieux = remplacer_non_renseigne(df_lieux)
#traiter valeur invalide de df_lieux
df_lieux = traiter_valeurs_speciales(df_lieux)

In [8]:
# nombre disctincts de valeurs pour la colonne nombre de voies
print(df_lieux['nombre_de_voies'].groupby(df_lieux['nombre_de_voies']).size().to_frame('count').reset_index())
print("type de la colonne nombre_de_voies avant transformation :", df_lieux['nombre_de_voies'].dtype)

df_lieux['nombre_de_voies'] = (
    pd.to_numeric(df_lieux['nombre_de_voies'], errors='coerce')
    .fillna(0)
    .astype(int)
)
print("type de la colonne nombre_de_voies après transformation :", df_lieux['nombre_de_voies'].dtype)
df_lieux['nombre_de_voies'].groupby(df_lieux['nombre_de_voies']).size().to_frame('count').reset_index()

   nombre_de_voies   count
0                1   58093
1               10     431
2               11      83
3               12      57
4               13      11
5                2  267456
6                3   36455
7                4   46970
8                5    5242
9                6    9857
10               7     738
11               8    2239
12               9     251
13   Non renseigné   45570
type de la colonne nombre_de_voies avant transformation : object
type de la colonne nombre_de_voies après transformation : int64


,nombre_de_voies,count
0,0,48028
1,1,58093
2,2,267456
3,3,36455
4,4,46970
5,5,5242
6,6,9857
7,7,738
8,8,2239
9,9,251


In [9]:
#harmoniser la colonne categorie_route
#Codes/Libellés : aligner (ex. Catégorie route : gérer 7 vs texte via table de correspondance ONISR).

print(df_lieux.groupby('categorie_route').size().reset_index(name='counts').sort_values(by='counts', ascending=False))
df_lieux.groupby('categorie_route').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
mapping_categorie = {
'1': 'Autoroute', 
'2': 'Route Nationale',
'3': 'Route Départementale',
'4': 'Voie Communale',
'5': 'Hors réseau public',
'6': 'Parc de stationnement ouvert à la circulation publique',
'9': 'Autre',
'Autoroute': 'Autoroute', 
'Route Nationale': 'Route Nationale',
'Route Départementale': 'Route Départementale',
'Voie Communale': 'Voie Communale',
'Hors réseau public': 'Hors réseau public',
'Parc de stationnement ouvert à la circulation publique': 'Parc de stationnement ouvert à la circulation publique',
'Autre': 'Autre'
}
df_lieux = harmonise_colonne_df(df_lieux,'categorie_route',mapping_categorie, 'Autre')
print('\napres harmonisation:')
print(df_lieux.groupby('categorie_route').size().reset_index(name='counts').sort_values(by='counts', ascending=False))


                                     categorie_route  counts
6                                     Voie Communale  230977
4                               Route Départementale  161026
1                                          Autoroute   41951
5                                    Route Nationale   30360
7                                              autre    5800
3  Parc de stationnement ouvert à la circulation ...    3334
0                                                  7    1692
2                                 Hors réseau public     771

apres harmonisation:
                                     categorie_route  counts
6                                     Voie Communale  230977
4                               Route Départementale  161026
0                                          Autoroute   41951
5                                    Route Nationale   30360
1                                              Autre    7492
3  Parc de stationnement ouvert à la circulation ...    3334
2 

In [10]:
#Fermeture des connexions actives à la base de données
admin_engine = get_engine(ADMIN_DB_URL)
show_open_connections(admin_engine, DB_NAME)
close_all_connections(admin_engine, DB_NAME)
show_open_connections(admin_engine, DB_NAME)


INFO:__main__:Toutes les connexions à 'db_accident' ont été fermées (sauf la session actuelle).
INFO:__main__:Engine SQLAlchemy libéré — plus aucune connexion active.


(3686, 'postgres', 'db_accident', 'idle', 'ROLLBACK')
